In [4]:
!pip install langchain
!pip install gpt4all
!pip install pypdf
!pip install faiss-cpu

     -------------------------------------- 803.6/803.6 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 365.5/365.5 kB 7.6 MB/s eta 0:00:00
  Using cached dataclasses_json-0.6.3-py3-none-any.whl (28 kB)
     -------------------------------------- 230.3/230.3 kB 7.1 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.3 which is incompatible.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.36.0.dev0 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


     ---------------------------------------- 1.6/1.6 MB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 49.3/49.3 kB 2.4 MB/s eta 0:00:00
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached marshmallow-3.20.2-py3-none-any.whl (49 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.1
    Uninstalling aiohttp-3.8.1:
      Successfully uninstalled aiohttp-3.8.1



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
from langchain_community.document_loaders import PyPDFLoader
#text spliter
from langchain.text_splitter import RecursiveCharacterTextSplitter
#embeding function
from langchain_community.embeddings import GPT4AllEmbeddings
#vector database
from langchain.vectorstores import FAISS
#llm from anyscle
from langchain_community.llms import Anyscale
#chain
from langchain.chains import RetrievalQA
#memory
from langchain.memory import ConversationBufferMemory
#prompt templatte
from langchain.prompts import PromptTemplate

In [21]:
#Loading pdf
#Loading pdf
def data_loader(pdf):
  pdf =PyPDFLoader(pdf).load()
  return pdf

In [22]:
# Spliting loaded documents
def convert_to_chunks(data):
    spliter = RecursiveCharacterTextSplitter(
        chunk_size = 600,
        chunk_overlap = 60)
    docs  = spliter.split_documents(data)
    return docs

In [8]:
len(docs)

94

In [28]:
#importing and defining embedding model, we are using opensource 'GPT4AllEmbeddings'
#embedding and storing in a vector database
def vector_store(docs):
    gpt4all_embd = GPT4AllEmbeddings()
    vdb = FAISS.from_documents(docs, gpt4all_embd)
    return vdb

In [29]:
#Defining LLm model
#Anyscale llm-llama model 70B
ANYSCALE_API_BASE = "https://api.endpoints.anyscale.com/v1"
ANYSCALE_API_KEY = "esecret_pmx42up5fwnqdlwrffrgpvin5u"
ANYSCALE_MODEL_NAME = "meta-llama/Llama-2-70b-chat-hf"

os.environ["ANYSCALE_API_BASE"] = ANYSCALE_API_BASE
os.environ["ANYSCALE_API_KEY"] = ANYSCALE_API_KEY

In [30]:
#defining LLm
llm = Anyscale(model_name=ANYSCALE_MODEL_NAME, temperature=0.1)

## prompt template and chain function

In [31]:
def chain_function(vdb):
    prompt_template = """
    you are a friendly question answering bot\
    you are expected to ask questions by users you have to answer to users question in a friendly way\
    Do not  do not hallucinate, generate an answer based on this context only.\
    Directly answer to the user question.\
    CONTEXT: {context}

    QUESTION: {question}"""


    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    chain_type_kwargs = {"prompt": PROMPT}
    chain = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=vdb.as_retriever(),
                                input_key="query",
                                return_source_documents=True,
                                chain_type_kwargs=chain_type_kwargs)
    return chain

In [ ]:
if __name__ == "__main__":
    data = data_loader("C:\\Users\\IHSAN B P\\Desktop\\ibp\\gen ai assessment\\48lawsofpower.pdf")
    docs = convert_to_chunks(data)
    vdb = vector_store(docs)
    chain = chain_function(vdb)
    response = chain("Can you give me an example from history where the enemy was crushed totally from the book?")
    print(response['result'])

# console

In [33]:
#console where you can aske your question and you will get the asnwer
if __name__ == "__main__":
    # Load and process the data
    data = data_loader("C:\\Users\\IHSAN B P\\Desktop\\ibp\\gen ai assessment\\48lawsofpower.pdf")
    docs = convert_to_chunks(data)
    vdb = vector_store(docs)

    # Create a chain function instance
    chain = chain_function(vdb)

    # Console loop to ask questions
    while True:
        question = input("Ask a question (type 'exit' to quit): ")
        if question.lower() == 'exit':
            break
        response = chain(question)
        print(response['result'])

Ask a question (type 'exit' to quit): exit


# web app


In [34]:
!pip install streamlit

     ---------------------------------------- 8.4/8.4 MB 10.1 MB/s eta 0:00:00
     ------------------------------------- 196.4/196.4 kB 12.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 4.8/4.8 MB 17.0 MB/s eta 0:00:00
     ------------------------------------- 996.9/996.9 kB 21.0 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB ? eta 0:00:00
     ------------------------------------- 346.6/346.6 kB 22.4 MB/s eta 0:00:00
  Using cached smmap-5.0.1-py3-none-any.whl (24 kB)


In [35]:
import streamlit as st

In [36]:
def get_answer(question):
    # Use the chain function to get the response for the given question
    response = chain(question)

    # Extract the result from the response
    # Make sure this aligns with how your chain function structures its response
    answer = response['result'] if 'result' in response else "No answer found."

    return answer

In [37]:
# Streamlit UI components
st.title('48 Laws of Power Q&A')

question = st.text_input("Ask a question:")

if st.button('Submit'):
    answer = get_answer(question)
    st.write("Answer:", answer)

2024-01-26 23:19:34.879 
  command:

    streamlit run C:\Users\IHSAN B P\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [38]:
import streamlit as st

# Include your helper functions and the get_answer function here

def main():
    st.title('48 Laws of Power Q&A App')

    # Text input for the question
    question = st.text_input("Enter your question about 'The 48 Laws of Power':")

    # Button to submit the question
    if st.button('Get Answer'):
        if question:
            answer = get_answer(question)
            st.text("Answer:")
            st.write(answer)
        else:
            st.write("Please enter a question.")

if __name__ == "__main__":
    main()


In [44]:
!pip install flask


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:


from flask import Flask, jsonify, request
import threading

# Include all your helper functions and imports

# Global initialization
data = data_loader("C:\\Users\\IHSAN B P\\Desktop\\ibp\\gen ai assessment\\48lawsofpower.pdf")
docs = convert_to_chunks(data)
vdb = vector_store(docs)
chain = chain_function(vdb)

app = Flask(__name__)

# Flask app route and functions
...


Ellipsis

In [49]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        question = request.form.get('question')
        response = chain(question)
        answer = response['result']
        return jsonify({'answer': answer})
    return '''
        <form method="post">
            Question: <input type="text" name="question"><br>
            <input type="submit" value="Submit">
        </form>
    '''


In [50]:
def run_app():
    app.run(port=5000, use_reloader=False)

flask_thread = threading.Thread(target=run_app)
flask_thread.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jan/2024 23:44:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 23:44:40] "POST / HTTP/1.1" 200 -
